# Visualizing Output Data -- WaveMoL
---

Cactus output files can be confusing to a new user. The aim of this tutorial is to provide you with hands-on practice working with output files and visualizing their data. To do this, we will run a simple simulation and then visualize its output.

**Simulation:** we will use the WaveMol example as our test case. The source code is located in `Cactus/arrangements/CactusExamples/WaveMoL`. We will not be writing any source code, creating new thorns, re-compiling Cactus, or anything like that. We will simply modify the WaveMoL parfile (below) so that the simulation generates examples of many of the kinds of output files that you will encounter. If you have a working installation of the Einstein Toolkit, and if you have the necessary Python libraries installed, everything in this notebook should "just work".

**Visualization:** we will use Python's matplotlib graphics library to visualize our simulation data. This is not necessarily the best choice, but it is convenient, especially for a Jupyter notebook-based tutorial such as this. At the very least, visualization in Python can be a useful preliminary diagnostic, before moving to a more advanced tool such as VisIt.

**Python Libraries:** parts of this notebook require NumPy, Matplotlib, and H5Py. Verify that these are installed on your machine before proceding.

**Further Reading:** there is a lot of excellent I/O documentation for Cactus. For example, see the documentation for the 
[IOUtil](http://einsteintoolkit.org/thornguide/CactusBase/IOUtil/documentation.html), 
[IOBasic](http://einsteintoolkit.org/thornguide/CactusBase/IOBasic/documentation.html), 
[IOASCII](http://einsteintoolkit.org/thornguide/CactusBase/IOASCII/documentation.html), 
and 
[IOHDF5](http://einsteintoolkit.org/thornguide/CactusPUGHIO/IOHDF5/documentation.html)
thorns. A condensed version of that material is presented here.

**What this tutorial is NOT:** there are two important topics that this tutorial does not get into.

1. Carpet output. The WaveMoL thorn uses a uniform grid and the corresponding PUGH driver. It is very likely that the applications you are interested in will require the Carpet driver instead of PUGH, which allows for mesh refinement. The output files generated by Carpet are slightly more complicated. Unfortunately this tutorial will not examine those output files.


2. Output from multiple processors. This tutorial is meant as an introduction to Cactus output. The imagined audience is someone working on a laptop or desktop workstation. As such, it is assumed here that you are running this notebook on a single processor, not a cluster. How to handle output from multiple processors adds a layer of complexity that we will not get into here.

Hopefully both of these important topics will be addressed in a future tutorial.

## 0. Preliminaries

First, specify the paths on your machine to the Cactus directory, simulations directory, and the directory where you want to store the edited copy of the WaveMoL parfile.

In [ ]:
## Linux example:
#cactus_dir = '/home/ejwest/ETK/Cactus'
#sims_dir   = '/home/ejwest/ETK/simulations'
#par_dir    = '/home/ejwest/ETK/parfiles'

## Windows example:
#cactus_dir = 'C:\\Users\\EJWest\\ETK\\Cactus'
#sims_dir   = 'C:\\Users\\EJWest\\ETK\\Simulations'
#par_dir    = 'C:\\Users\\EJWest\\ETK\\Parfiles'

## Fill in paths to directories:
import os
home_dir = os.environ["HOME"]
cactus_dir = os.path.join(home_dir, "Cactus") #<-- Path to Cactus directory
sims_dir   = os.path.join(home_dir, "simulations") #<-- Path to simulations directory
par_dir    = os.path.join(cactus_dir, "parfiles") #<-- Path to parfile directory

os.makedirs(par_dir, exist_ok=True)

## 1. Edit Parfile

Next, we create a modified copy of the parfile in the directory that you specified above. The parts of the file that have to do with controlling output have commented headers. For the sake of illustation, here we are generating all three kinds of output files: xgraph-style ASCII, gnuplot-style ASCII, and HDF5.

In [ ]:
%%writefile {par_dir}/wave_mol.par
Cactus::cctk_itlast = 100

ActiveThorns = "CoordBase CartGrid3D SymBase Boundary"
ActiveThorns = "PUGHReduce PUGH PUGHSlab"
Grid::domain = "full"
Grid::type = "byspacing"
Grid::avoid_origin = "no"
driver::global_nx = 51
driver::global_ny = 51
driver::global_nz = 51
Grid::dxyz = 0.02
driver::ghost_size = 1

ActiveThorns = "NaNChecker"
ActiveThorns = "LocalReduce"

ActiveThorns = "MoL"
MethodOfLines::ode_method = "icn" #default: icn
        
ActiveThorns = "IDWaveMoL"
IDWaveMoL::initial_data = "gaussian"
IDWaveMoL::amplitude = 1.0 #default: 1.0
IDWaveMoL::radius = 0.0 #default: 0
IDWaveMoL::sigma = 0.1 #default: 0.1
IDWaveMoL::centrex = 0.0 #default: 0
IDWaveMoL::centrey = 0.0 #default: 0
IDWaveMoL::centrez = 0.0 #default: 0
IDWaveMoL::kx = 0.0 #default: 0
IDWaveMoL::ky = 0.0 #default: 0
IDWaveMoL::kz = 0.0 #default: 0
IDWaveMoL::offsett = 0.0 #default: 0
IDWaveMoL::offsetx = 0.0 #default: 0
IDWaveMoL::offsety = 0.0 #default: 0
IDWaveMoL::offsetz = 0.0 #default: 0
IDWaveMoL::slopet = 0.0 #default: 0
IDWaveMoL::slopex = 0.0 #default: 0
IDWaveMoL::slopey = 0.0 #default: 0
IDWaveMoL::slopez = 0.0 #default: 0

ActiveThorns = "WaveMoL"
WaveMoL::bound = "radiation"

ActiveThorns = "Time"
Time::timestep_method = "courant_static"
Time::dtfac = 0.5 #default: 0.5

##--------------------------------------------------------##
## IOUtil                                                 ##
##--------------------------------------------------------##
ActiveThorns = "IOUtil"
IO::out_mode = "proc"
IO::out_dir = $parfile

##--------------------------------------------------------##
## IOBasic - provides I/O methods for outputting scalar   ##
## values in ASCII format into files and for printing     ## 
## them as runtime information to screen                  ##
##--------------------------------------------------------##
ActiveThorns = "IOBasic"

## stdout
IOBasic::outInfo_every = 1
IOBasic::outInfo_vars  = "
        WaveMoL::phi 
        #WaveMoL::energy
"
IOBasic::outInfo_reductions = "minimum maximum"

## ASCII output for scalars
IOBasic::outScalar_every = 1
IOBasic::outScalar_vars  = "
        WaveMoL::phi
        #WaveMoL::energy
"
IOBasic::outScalar_style = "xgraph" #default
#IOBasic::outScalar_style = "gnuplot"

##--------------------------------------------------------##
## IOASCII - provides I/O methods for 1D, 2D, and 3D      ## 
## output of grid arrays and grid functions into files in ##
## ASCII format. The precise format is designed for       ##
## visualisation using the clients xgraph or gnuplot.     ##
##--------------------------------------------------------##
ActiveThorns = "IOASCII"

## ASCII output for 1D slices
IOASCII::out1D_every = 2
IOASCII::out1D_x     = yes
IOASCII::out1D_y     = yes
IOASCII::out1D_z     = yes
IOASCII::out1D_d     = yes       
IOASCII::out1D_vars  = "
        WaveMoL::phi
        #WaveMoL::energy
"
#IOASCII::out1D_style = "xgraph" #default
#IOASCII::out1D_style = "gnuplot f(x)"
IOASCII::out1D_style = "gnuplot f(t,x)"

## ASCII output for 2D slices
IOASCII::out2D_every = 2
IOASCII::out2D_xyplane_z = 0.0
IOASCII::out2D_yzplane_x = 0.0
IOASCII::out2D_xzplane_y = 0.0
IOASCII::out2D_vars  = "
        WaveMoL::phi
        #WaveMoL::energy
"
#IOASCII::out2D_style = "gnuplot f(x,y)" #default
IOASCII::out2D_style = "gnuplot f(t,x,y)"

##--------------------------------------------------------##
## IOHDF5 - provides an I/O method to output variables in ##
## HDF5 file format. It also implements checkpointing and ##
## recovery functionality using HDF5.                     ##
##--------------------------------------------------------##
ActiveThorns = "IOHDF5"
IOHDF5::out_every            = 5
IOHDF5::out_vars             = "
        WaveMoL::phi
        #WaveMoL::energy
        Grid::Coordinates{out_every=1000000000} #we only want this to print once
"
IOHDF5::checkpoint                  = no

## 2. Run Simulation

In [ ]:
%rm -rf {sims_dir}/wave_mol

In [ ]:
%cd {cactus_dir}
!./simfactory/bin/sim create-run wave_mol --parfile={par_dir}/wave_mol.par --procs=2

## 3. Analyze Output

Now move to the output directory for the WaveMoL simulation that we just ran. (This assumes you entered the correct path at the beginning of the notebook.)

In [ ]:
%cd {sims_dir}/wave_mol/output-0000/wave_mol

Take a look at the contents of this directory.

In [ ]:
%ls

There are three output file format options in Cactus: xgraph-style ASCII (file extension .xg), gnuplot-style ASCII (file extension .asc), or HDF5 (file extension .h5). Notice that all three types of files are present in this example, by design. 

In the parfile above, we selected "scalars" to be written in xgraph-style ASCII format (see the `IOBasic::outScalar` lines). Scalars are quantities that have one value per time step. There are four scalar output files here:

- `phi_maximum.xg` (maximum value of phi over the entire grid, at each time step)
- `phi_minimum.xg` (minimum value of phi over the entire grid, at each time step)
- `phi_norm1.xg` (L1 norm of phi over the entire grid, at each time step)
- `phi_norm2.xg` (L2 norm of phi over the entire grid, at each time step)

In the parfile we selected 1D ASCII output to be written in gnuplot-style format (see the `IOASCII::out1D` lines). 1D output is usually the value of some function along a 1D slice of the computational domain. There are four instances of 1D ASCI output files here:

- `phi_3D_diagonal.asc` (phi along a 3D diagonal of the grid, at each time step)
- `phi_x_[25][25].asc` (phi along the x-axis of the grid, at each time step)
- `phi_y_[25][25].asc` (phi along the y-axis of the grid, at each time step))
- `phi_z_[25][25].asc` (phi along the z-axis of the grid, at each time step)

In the parfile we selected 2D ASCII output to be written in gnuplot-style format (see the `IOASCII::out2D` lines). 2D output is usually the value of some function along a 2D slice of the computational domain. There are three instances of 2D ASCI output files here:

- `phi_xy_[25].asc` (phi along the xy-plane of the grid, at each time step)
- `phi_xz_[25].asc` (phi along the xz-plane of the grid, at each time step)
- `phi_yz_[25].asc` (phi along the yz-plane of the grid, at each time step)

Finally, we selected HDF5 output to also be written (see the `IOHDF5` lines). There are five instances of HDF5 output files here:

- `phi.h5` (phi over the full 3D grid, at each time step)
- `r.h5` (???, at each time step)
- `x.h5` (x coordinates, in 3D meshgrid form, at each time step)
- `y.h5` (y coordinates, in 3D meshgrid form, at each time step)
- `z.h5` (z coordinates, in 3D meshgrid form, at each time step)

We will examine each of these types of output files in turn, starting with the simplest case of scalar output and finishing with the most difficult case of HDF5 output.

## 4. Plotting ASCII Scalar Output Data

Scalar output is the simplest to understand, and so this serves as a warm-up case. Scalars usually refer to some global property over the grid at each time step. These are refered to as "reductions" in the Cactus documentation (because information over the entire grid is reduced to a single number at each time step). An example is the average value of a grid function, over the grid, at a given time step. Other examples are the various norms (L1, L2, etc) of a grid function, over the grid, at a given time step.

Scalar output is very simple. It contains two columns: time step and the scalar value at that time step. This is true regardless of whether the output is xgraph-style ASCII (.xg) or gnuplot-style ASCII (.asc). 

Unless you make changes, the parfile above is setup to print xgraph-style ASCII files for scalars. You can repeat the following steps for gnuplot-style ASCII by modifying the parfile above, re-running the simulation, and then toggle the commented lines below to switch from xgraph to gnuplot style output (i.e., comment out the lines that immediately follow "## xgraph" and uncomment lines that immediately follow "## gnuplot").

Let's list the xgraph-style ASCII (.xg) files that we have to choose from.

In [ ]:
## xgraph
%ls *.xg

## gnuplot
#%ls *.asc

Choose one of these files to work with. Here we will choose the `phi_norm2` file. To see the structure of the contents of the file, we print the first 10 lines.

In [ ]:
### choose a file ###

## xgraph
file1 = 'phi_norm2.xg'

## gnuplot
#file1 = 'phi_norm2.asc'

## print first 10 lines of file
!head -n 10 {file1}

Notice that xgraph-style files use quotation marks (") to begin comments. Here we see the structure as advertised above. After a few lines of comments, the file contains two columns, with the structure

- Column 1: time step
- Column 2: scalar data

Next we assign those data to a numpy array.

In [ ]:
## import numpy library
import numpy as np

## xgraph
data1 = np.loadtxt(file1, comments='"')

## gnuplot
#data1 = np.loadtxt(file1, comments='#')

## print data to screen (optional)
print(data1)

Then for the sake of code-readability, we can assign meaningful names to different parts of the data.

In [ ]:
## get time steps
time_steps = data1[:,0]  #every row, first col

## get scalar values
phi_norm2  = data1[:,1]  #every row, second col

## print data to screen (optional)
print('time_steps = ', time_steps)
print('phi_norm2 = ', phi_norm2)

And finally we can plot the scalar data versus time.

In [ ]:
## set graphics backend
%matplotlib notebook
## import graphics libraries
import matplotlib.pyplot as plt

## plot
plt.plot(time_steps, phi_norm2, color='red', linewidth=2)
## NOTE: you can use LaTeX in your labels by prefixing with r.
## The r tells matplotlib to "render" the string as a LaTeX string.
plt.title(r'$L_{2}$ norm of the grid function $\phi(x,y,z)$')  
plt.xlabel('time')
plt.ylabel(r'$||\phi||_{2}$')
plt.show()

### Exercises

1) Plot the maximum value of phi as a function of time.

2) Modify the parfile above to print gnuplot-style ASCII output for scalars. Then plot the L2 norm of phi as a function of time.

3) Modify the parfile above to print scalar output for energy. Then plot the L2 norm for energy as a function of time.

## 5. Plotting ASCII 1D Output Data

A 1D slice returns the value of some grid function along a ray in the computational domain. An example would be some evolved variable along the x-axis at each time step.

1D output data is slightly more complicated than the scalar case. The form of the output depends on the style that you specify in the parfile:

- `IOASCII::out1D_style = "xgraph"`$\rightarrow$ two columns: x, f(x)
- `IOASCII::out1D_style = "gnuplot f(x)"` $\rightarrow$ two columns: x, f(x)
- `IOASCII::out1D_style = "gnuplot f(t,x)"` $\rightarrow$ three columns: t, x, f(t,x)

Unless you make changes, the parfile above is setup to print gnuplot-style ASCII files for 1D slices using the "f(t,x)" format.

Let's list the gnuplot-style ASCII (.asc) files that we have to choose from.

In [ ]:
## xgraph
#%ls *.xg

## gnuplot
%ls *.asc

Choose one of these files to work with. Here we will choose the `phi_x` file. To see the structure of the contents of the file, we print the first 10 lines.

In [ ]:
### choose a file ###

## xgraph
#file2 = 'phi_x_[25][25].xg'

## gnuplot
file2 = 'phi_x_[25][25].asc'

## print first 10 lines of file
!head -n 10 {file2}

Notice that gnuplot-style files use the hash symbol (#) to begin comments. After a few lines of comments, the file contains three columns, with the structure

- Column 1: time step 
- Column 2: grid values (slice coordinate)
- Column 3: grid function values

Time steps are printed in the first column because "gnuplot f(t,x)" was selected in the parfile. The other options would have been to use "gnuplot f(x)" or "xgraph", in which case only two columns would appear (which appear here as columns 2 and 3).

Next we assign the data to a numpy array.

In [ ]:
## import numpy library
import numpy as np

## xgraph
#data2 = np.loadtxt(file2, comments='"')

## gnuplot
data2 = np.loadtxt(file2, comments='#')

## print data to screen (optional)
print(data2)

Next we would like to assign meaningful names to different parts of the data. However, this time things are slightly more complicated by the block structure of the data, which schematically looks something like this

\begin{align}
\text{block 1}\left\{
  \begin{array}{ccc}
  \tt{t1} & \tt{x1} & \tt{f(t1, x1)} \\
  \tt{t1} & \tt{x2} & \tt{f(t1, x2)} \\
  \tt{t1} & \tt{x3} & \tt{f(t1, x3)} \\
  \end{array}\right.
\end{align}

\begin{align}
\text{block 2}\left\{
  \begin{array}{ccc}
  \tt{t2} & \tt{x1} & \tt{f(t2, x1)} \\
  \tt{t2} & \tt{x2} & \tt{f(t2, x2)} \\
  \tt{t2} & \tt{x3} & \tt{f(t2, x3)} \\
  \end{array}\right.
\end{align}

\begin{align}
\text{block 3}\left\{
  \begin{array}{ccc}
  \tt{t3} & \tt{x1} & \tt{f(t3, x1)} \\
  \tt{t3} & \tt{x2} & \tt{f(t3, x2)} \\
  \tt{t3} & \tt{x3} & \tt{f(t3, x3)} \\
  \end{array}\right.
\end{align}

One way to handle this is as follows.

1. Create a 1D array of time steps by scanning the entire first column and keeping the unique elements.
2. Create a 1D array of spatial grid values by scanning the entire second column and keeping the unique elements. 
3. Create a 2D array of grid function values: each row corresponding to a different time step.

NOTE: step 2 is a compromise of efficiency for convenience. Since we are dealing with a fixed grid, we don't need to scan the entire second column; we could just scan the second column of the first block. But it's easier to just scan the entire second column. Doing things in a more clever way is left as an exercise to the interested reader! 

In [ ]:
### gnuplot, f(t,x) ###

## identify columns
t_col = 0
x_col = 1
f_col = 2

## step 1: get unique time steps
time_steps = np.unique(data2[:,t_col])
tdim = len(time_steps)

## step 2: get unique spatial grid values
xvals = np.unique(data2[:,x_col])
xdim = len(xvals)

## step 3: get data
phi_x = np.zeros((tdim, xdim))
for i in range(tdim):
    ## get time step
    tstep = time_steps[i]
    ## fetch corresponding data
    tsdata = data2[data2[:,t_col]==tstep, :] 
    ## save data to arrays
    phi_x[i,:] = tsdata[:, f_col]

## print data to screen (optional)
print('time_steps = ', time_steps)
print('xvals =', xvals)
print('phi_x = ', phi_x)

The following cell contains the analogous commands if you use "gnuplot f(x)" style output instead. (Try it!)

In [ ]:
### gnuplot, f(x) ###

## identify columns
#x_col = 0
#f_col = 1

## step 1: get unique time steps
#import re
#fh = open(file2).read()
#time_steps = []
#for line in re.findall('#Time =.*', fh):
#    tstep = float(line.split()[2])
#    time_steps.append(tstep)
#f.close()
#time_steps = np.array(time_steps)
#time_steps
#tdim = len(time_steps)

## step 2: get unique spatial grid values
#xvals = np.unique(data2[:,x_col])
#xdim = len(xvals)

## step 3: get data
#phi_x = np.zeros((tdim, xdim))
#for i in range(tdim):
#    ## get grid value
#    xstep = xvals[i]
#    ## fetch corresponding data
#    xsdata = data2[data2[:,x_col]==xstep, :] 
#    ## save data to arrays
#    phi_x[:,i] = xsdata[:, f_col]

## print data to screen (optional)
#print('time_steps = ', time_steps)
#print('xvals =', xvals)
#print('phi_x = ', phi_x)

Now we are ready to plot the data in whatever way we like.

### Plot snapshot

In [ ]:
## set graphics backend
%matplotlib notebook
## import graphics libraries
import matplotlib.pyplot as plt

## figure properties
fig_width = 8
fig_height = 4
minval = np.min(phi_x)
maxval = np.max(phi_x)

## choose a time step
tstep = 1

## plot
fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_subplot(111)
ax.plot(xvals[:], phi_x[tstep,:], color='red', linewidth=2)
ax.set_ylim(minval, 1.1*maxval)
ax.set_title(r'$\phi$ along $x$ axis')
ax.set_xlabel('x')
ax.set_ylabel(r'$\phi$')
ax.text(0.35, 0.95, 'time = %.3f' % time_steps[tstep])
plt.show()

### Plot animation

In [ ]:
## import graphics libraries
import matplotlib.pyplot as plt
import matplotlib.animation as animation

## figure properties
fig_width = 8
fig_height = 4
minval = np.min(phi_x)
maxval = np.max(phi_x)

### initialize plot
plt.ioff()
fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_subplot(111)
ax.set_ylim(minval, 1.1*maxval)
time = time_steps[tstep]
ax.text(0.35, 0.95, 'time = %.3f' % time)
scene = ax.plot(xvals[:], phi_x[0,:], color='red', linewidth=2)

## update plot
def update_scene(frame_number, xvals, phi_x, scene):
    ax.cla()
    ax.set_ylim(minval, 1.1*maxval)
    time = time_steps[frame_number]
    ax.text(0.35, 0.95, 'time = %.3f' % time)
    scene = ax.plot(xvals[:], phi_x[frame_number,:], color='red', linewidth=2)

## make animation
fps=5
frames=np.arange(0, len(time_steps), 1)
anim = animation.FuncAnimation(
    fig, update_scene, frames, fargs=(xvals, phi_x, scene), 
    interval=1000/fps, blit=False, repeat=False)
plt.close()

## play animation
from IPython.display import HTML
HTML(anim.to_html5_video())  #playback option 1
#HTML(anim.to_jshtml())       #playback option 2

### Exercises

1) Plot phi_z as a function of time.

2) Modify the parfile to use the "gnuplot f(x)" style for 1D output and then replot phi_x as a function of time.

3) Modify the parfile to use the "xgraph" style for 1D output and then replot phi_x as a function of time.

## 6. Plotting ASCII 2D Output Data

A 2D slice returns the value of some grid function along a plane in the computational domain. An example would be some evolved variable over the xy-plane at each time step.

The output for 2D slices is more complicated still. In this case there is no xgraph-style ASCII option. But as with 1D slices, there are two gnuplot-style options to choose from in the parfile:

- `IOASCII::out2D_style = "gnuplot f(x,y)"` $\rightarrow$ three columns: x, y, f(x,y)
- `IOASCII::out2D_style = "gnuplot f(t,x,y)"` $\rightarrow$ four columns: t, x, y, f(t,x,y)

Unless you make changes, the parfile above is setup to print gnuplot-style ASCII files for 2D slices using the "f(t,x,y)" format.

Re-list the gnuplot-style ASCII files that we have to choose from.

In [ ]:
%ls *.asc

Choose a file to work with. Here we will take the `phi_xy` file, which gives the grid function phi on the xy-plane, at each time step.

In [ ]:
### choose a file ###
file3 = 'phi_xy_[25].asc'

## print first 10 lines
!head -n 10 {file3}

After a few lines of comments, the file contains four columns, with the structure

- Column 1: time step 
- Column 2: grid values (first slice coordinate)
- Column 3: grid values (second slice coordinate)
- Column 4: grid function values

Time steps are printed in the first column because "gnuplot f(t,x,y)" was selected in the parfile. The other option would have been to use "gnuplot f(x,y)", in which case only three columns would appear (which appear here as columns 2-4). (Try it!)

Next we assign the data to a numpy array.

In [ ]:
## import numpy library
import numpy as np

## get data
data3 = np.loadtxt(file3, comments='#')

## print data to screen (optional)
print(data3)

Next we would like to assign meaningful names to different parts of the data. This time the structure schematically looks something like this

\begin{align}
\text{block 1}\left\{
  \begin{array}{cccc}
  \tt{t1} & \tt{x1} & \tt{y1} & \tt{f(t1, x1, y1)} \\
  \tt{t1} & \tt{x1} & \tt{y2} & \tt{f(t1, x1, y2)} \\
  \tt{t1} & \tt{x1} & \tt{y3} & \tt{f(t1, x1, y3)} \\
  \end{array}\right.
\end{align}

\begin{align}
\text{block 2}\left\{
  \begin{array}{cccc}
  \tt{t1} & \tt{x2} & \tt{y1} & \tt{f(t1, x2, y1)} \\
  \tt{t1} & \tt{x2} & \tt{y2} & \tt{f(t1, x2, y2)} \\
  \tt{t1} & \tt{x2} & \tt{y3} & \tt{f(t1, x2, y3)} \\
  \end{array}\right.
\end{align}

\begin{align}
\text{block 3}\left\{
  \begin{array}{cccc}
  \tt{t1} & \tt{x3} & \tt{y1} & \tt{f(t1, x3, y1)} \\
  \tt{t1} & \tt{x3} & \tt{y2} & \tt{f(t1, x3, y2)} \\
  \tt{t1} & \tt{x3} & \tt{y3} & \tt{f(t1, x3, y3)} \\
  \end{array}\right.
\end{align}

\begin{align}
\text{block 4}\left\{
  \begin{array}{cccc}
  \tt{t2} & \tt{x1} & \tt{y1} & \tt{f(t2, x1, y1)} \\
  \tt{t2} & \tt{x1} & \tt{y2} & \tt{f(t2, x1, y2)} \\
  \tt{t2} & \tt{x1} & \tt{y3} & \tt{f(t2, x1, y3)} \\
  \end{array}\right.
\end{align}

\begin{align}
\text{block 5}\left\{
  \begin{array}{cccc}
  \tt{t2} & \tt{x2} & \tt{y1} & \tt{f(t2, x2, y1)} \\
  \tt{t2} & \tt{x2} & \tt{y2} & \tt{f(t2, x2, y2)} \\
  \tt{t2} & \tt{x2} & \tt{y3} & \tt{f(t2, x2, y3)} \\
  \end{array}\right.
\end{align}

\begin{align}
\text{block 6}\left\{
  \begin{array}{cccc}
  \tt{t2} & \tt{x3} & \tt{y1} & \tt{f(t2, x3, y1)} \\
  \tt{t2} & \tt{x3} & \tt{y2} & \tt{f(t2, x3, y2)} \\
  \tt{t2} & \tt{x3} & \tt{y3} & \tt{f(t2, x3, y3)} \\
  \end{array}\right.
\end{align}

We can handle this is as follows.

1. Create a 1D array of time steps by scanning the entire first column and keeping the unique elements.
2. Create a 1D array of spatial grid values by scanning the entire second column and keeping the unique elements.
3. Create a 1D array of spatial grid values by scanning the entire third column and keeping the unique elements.
4. Create a 3D array of grid function values: each row corresponding to a different time step.

NOTE: as before, we are trading efficiency for convenience by scanning the entire second and third columns (steps 2 and 3). The interested reader is challenged to come up with a better approach!

In [ ]:
### gnuplot, f(t,x,y) ###

## identify columns
t_col = 0
x_col = 1
y_col = 2
f_col = 3

## step 1: get unique time steps
time_steps = np.unique(data3[:,t_col])
tdim = len(time_steps)

## step 2: get unique spatial grid values
xvals = np.unique(data3[:,x_col])
xdim = len(xvals)

## step 3: get unique spatial grid values
yvals = np.unique(data3[:,y_col])
ydim = len(yvals)

## step 4: get data
phi_xy = np.zeros((tdim, xdim, ydim))
for n in range(tdim):
    ## get time step
    tstep = time_steps[n]
    ## fetch corresponding data
    tsdata = data3[data3[:,t_col]==tstep, :]
    for i in range(xdim):
        ## get grid value
        xval = xvals[i]
        ## fetch corresponding data
        xsdata = tsdata[tsdata[:,x_col]==xval, :]
        ## save data to arrays
        phi_xy[n,i,:] = xsdata[:, f_col]

## print data to screen (optional)
print('time_steps = ', time_steps)
print('xvals =', xvals)
print('yvals =', yvals)
#print('phi_xy = ', phi_xy)

Now that the data is extracted into numpy arrays, we can visualize them in various ways. Here we will illustrate a couple of examples: we will make a color map using imshow() and a surface plot using plot_surface(). In addition, as above, we may either create a snapshot of the solution or an animation.

### Color map snapshot

In [ ]:
### plot snapshot using imshow() ###

## set graphics backend
%matplotlib notebook
## import graphics libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm

## figure properties
cmap = cm.gist_rainbow
fig_width = 8
fig_height = 4
minval = np.min(phi_xy)
maxval = np.max(phi_xy)

## choose a time step
tstep = 3

## plot
time = time_steps[tstep]
fig = plt.figure(figsize=(fig_width, fig_height))
fig.suptitle('time = %.3f' % time)
ax = fig.add_subplot(111)
zi = phi_xy[tstep,:,:]
ax.imshow(zi, cmap=cmap, clim=(minval, maxval), origin='lower')
plt.show()

### Color map animation

In [ ]:
### animate using imshow() ###

## set graphics backend
%matplotlib notebook
## import graphics libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation

## figure properties
cmap = cm.gist_rainbow
fig_width = 8
fig_height = 4
minval = np.min(phi_xy)
maxval = np.max(phi_xy)

## initialize plot
plt.ioff()
time = time_steps[tstep]
fig = plt.figure(figsize=(fig_width, fig_height))
fig.suptitle('time = %.3f' % time)
ax = fig.add_subplot(111)
zi = phi_xy[0,:,:]
scene = [ax.imshow(zi, cmap=cmap, clim=(minval, maxval), origin='lower')]

## update plot
def update_plot(frame_number, time_steps, phi_xy, scene):
    time = time_steps[frame_number]
    fig.suptitle('time = %.3f' % time)
    zi = phi_xy[frame_number,:,:]
    scene[0] = ax.imshow(zi, cmap=cmap, clim=(minval, maxval), origin='lower')

## make animation
fps=5
frames=np.arange(0, len(time_steps), 1)
anim = animation.FuncAnimation(
    fig, update_plot, frames, fargs=(time_steps, phi_xy, scene), 
    interval=1000/fps, blit=False, repeat=False)
plt.close()

## play animation
from IPython.display import HTML
HTML(anim.to_html5_video())  #playback option 1
#HTML(anim.to_jshtml())       #playback option 2

### Surface plot snapshot

In [ ]:
### plot snapshot using plot_surface() ###

## set graphics backend
%matplotlib notebook
## import graphics libraries
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

## figure properties
cmap = cm.gist_rainbow
fig_width = 8
fig_height = 4
minval = np.min(phi_xy)
maxval = np.max(phi_xy)

## choose a time step
tstep = 1

## plot
fig = plt.figure(figsize=(fig_width, fig_height))
time = time_steps[tstep]
fig.suptitle('time = %.3f' % time)
ax = fig.add_subplot(111, projection='3d')
ax.set_zlim(minval, maxval)
xi, yi = np.meshgrid(xvals, yvals)
zi = phi_xy[tstep,:,:]
ax.plot_surface(xi, yi, zi, cstride=1, rstride=1, cmap=cmap, clim=(minval, maxval))
plt.show()

### Surface plot animation

In [ ]:
### animate using plot_surface() ###

## set graphics backend
%matplotlib notebook
## import graphics libraries
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
import matplotlib.animation as animation

## figure properties
cmap = cm.gist_rainbow
fig_width = 8
fig_height = 4
minval = np.min(phi_xy)
maxval = np.max(phi_xy)

## initialize plot
plt.ioff()
fig = plt.figure(figsize=(fig_width, fig_height))
time = time_steps[0]
fig.suptitle('time = %.3f' % time)
ax = fig.add_subplot(111, projection='3d')
ax.set_zlim(minval, maxval)
xi, yi = np.meshgrid(xvals, yvals)
zi = phi_xy[0,:,:]
scene = [ax.plot_surface(xi, yi, zi, cstride=1, rstride=1, cmap=cmap, clim=(minval, maxval))]

## update plot
def update_plot(frame_number, time_steps, phi_xy, scene):
    time = time_steps[frame_number]
    fig.suptitle('time = %.3f' % time)
    scene[0].remove()
    zi = phi_xy[frame_number,:,:]
    scene[0] = ax.plot_surface(xi, yi, zi, cstride=1, rstride=1, cmap=cmap, clim=(minval, maxval))

## make animation
fps=5
frames=np.arange(0, len(time_steps), 1)
anim = animation.FuncAnimation(
    fig, update_plot, frames, fargs=(time_steps, phi_xy, scene), 
    interval=1000/fps, blit=False, repeat=False)
plt.close()

## play animation
from IPython.display import HTML
HTML(anim.to_html5_video())  #playback option 1
#HTML(anim.to_jshtml())       #playback option 2

In [ ]:
## save animation to file
anim_file = 'wave_mol'
#anim.save(anim_file + '.mp4', writer='ffmpeg', fps=fps) # requires ffmpeg
anim.save(anim_file + '.gif', writer='imagemagick', fps=fps)

### Exercises

1) Plot phi_yz as a function of time.

2) Modify the parfile to use the "gnuplot f(x,y)" style for 2D output and then replot phi_xy as a function of time.

3) Plot energy as a function of space and time.

## 7. Extracting Data from HDF5 Files 

HDF5 (Hierarchical Data Format 5) is a portable binary data format. As such, it is far more efficient to read and write than ASCII formats, and it is probably what you should use for visualizing large simulations. A nice tool for visualizing simulation data in HDF5 format is VisIt. However, the H5Py library also provides methods for interfacing with HDF5 files using Python. This is the approach we will take here. Once the data are extracted from the .h5 files, they can be visualized as normal using matplotlib.

In order to extract information from an HDF5 file, it helps to have at least some familiarity with how they are structured. This section will take a brief detour into exploring the basics of HDF5 so that the remainder of the tutorial doesn't seem like "black magic". If you just want to get to visualization and are ok with a little black magic, you can skip to the next section.

Before doing anything else, let's import the H5Py library.

In [ ]:
## import h5py library
import h5py

Now list the available HDF5 files.

In [ ]:
%ls *.h5

Then choose a file to work with. Here we will examine the `phi.h5` file.

In [ ]:
## create file handle
file4 = h5py.File('phi.h5', mode='r')

HDF5 files are not plain text files, so you can't read them using a text editor (try!). The basic elements of an HDF5 file are "groups" and "datasets". You can think of groups like directories or folders, and you can think of datasets like files. In addition, both groups and datasets have "attributes" that contain metadata about that object. Let's start by exploring the primary groups in our file. 

In [ ]:
## get (name, value) tuples for members of the root group
## (the root group, called '/', is the top-level group in the file)
list(file4.items())

Here we see that the root group contains two subgroups, `Cactus Parameters` and `Global Attributes`, in addition to several datasets with names of the form `WAVEMOL::phi timelevel 0 at iteration NN`. In the parfile above, we asked for HDF5 output every 5 time steps (see the `IOHDF5::out_every` line). Correspondingly, here we see that there is a dataset for every fifth time step (iteration). 

In general an HDF5 file could have a complicated group hierarchy with subgroups nested within subgroups, nested within subgroups, etc. Of the two groups in our file, one is empty (`Global Attributes` is listed as having zero "members", i.e., nothing in it) and one is not (`Cactus Parameters` is listed as having one member, which may be a dataset or another group). The following cell gives us a quick way to preview the entire hierarchy.

In [ ]:
## traverse the group hierarchy
file4.visit(lambda x: print(x))

From the above, we learn that the hierarchy in this case is not very deep or complicated. It should be clear that our data are in the `WAVEMOL::phi` datasets; there are no further datasets burried elsewhere in the hierarchy. The next cell extracts the first of these datasets in order to examine it further.

In [ ]:
## dictionary-like lookup (works for datasets as well as groups)
dset1 = file4['/WAVEMOL::phi timelevel 0 at iteration 0']
print(dset1)

The next cell prints some of the main attributes of the dataset.

In [ ]:
## get dataset attributes
print('dataset name is', dset1.name)
print('dataset type is', dset1.dtype)
print('dataset shape is', dset1.shape)

The next cell lists any additional attributes (metadata) that are stored with the dataset.

In [ ]:
## retreive (name, value) tuples of dataset attributes 
list(dset1.attrs.items())

Finally, the next cell illustrates how to retreive data in a dataset and store it in a NumPy array.

In [ ]:
## extract data from dataset using the '[...]' syntax
npdset1 = dset1[...]
print(npdset1)

The following cell verifies that this resulting object is indeed a NumPy array.

In [ ]:
type(npdset1)

At this point, we treat the data object as we would treat any other NumPy array. In the next section, we use what we have learned in this section to extract our data and then visualize them.

## 8. Plotting HDF5 2D Output Data

Now we return to the task of visualizing the data in an HDF5 (.h5) file. First import H5Py and NumPy.

In [ ]:
## import libraries
import numpy as np
import h5py

List the available HDF5 files.

In [ ]:
%ls *.h5

Choose a file to work with. Here we will examine the `phi.h5` file.

In [ ]:
## create file handle
phi_h5 = h5py.File('phi.h5', mode='r')

The following cell extracts the time steps from the metadata of the HDF5 file.

In [ ]:
## get time steps
time_steps = np.array([])
for member in phi_h5:
    ## find members with shape attributes
    ## (only datasets have shape attribute)
    if hasattr(phi_h5[member], 'shape'):
        ## get time of dataset
        ts = phi_h5[member].attrs['time']
        ## append time to time_steps
        time_steps = np.append(time_steps, ts)
    ## sort
    time_steps = np.sort(time_steps)

## print to screen (optional)
print("time_steps = ", time_steps)

In order to plot using `plot_surface()`, we need to construct an xy-meshgrid. The grid information is not stored in the `phi.h5` file, but instead is found in `x.h5`, `y.h5`, and `z.h5`. The following cell retrieves this information.

In [ ]:
## get 3D xdata, ydata, zdata
x_h5 = h5py.File("x.h5")
y_h5 = h5py.File("y.h5")
z_h5 = h5py.File("z.h5")

for member in x_h5:
    if hasattr(x_h5[member], 'shape'):
        xdata = x_h5[member][...]
        ## Stop! We only need to do this for one time 
        ## step, since we are dealing with a fixed grid!
        break 
for member in y_h5:
    if hasattr(y_h5[member], 'shape'):
        ydata = y_h5[member][...]
        ## Stop! We only need to do this for one time 
        ## step, since we are dealing with a fixed grid!
        break 
for member in z_h5:
    if hasattr(z_h5[member], 'shape'):
        zdata = z_h5[member][...]
        ## Stop! We only need to do this for one time 
        ## step, since we are dealing with a fixed grid!
        break 

## print to screen (optional)
print("xdata = ", xdata)
print("ydata = ", ydata)
print("zdata = ", zdata)

The next cell extracts the values of phi over the xy-grid at each time step.

In [ ]:
## get dimensions of time and space grids
tdim = len(time_steps)
xdim = len(xdata[0,0,:]) # indices: [z,y,x]
ydim = len(ydata[0,:,0]) # indices: [z,y,x]
zdim = len(ydata[:,0,0]) # indices: [z,y,x]

## get 3D phidata
phidata = np.zeros((tdim, xdim, ydim, zdim))
for i in range(tdim):
    ## get time step
    tstep = time_steps[i]
    ## find dataset with matching time step
    for member in phi_h5:
        if hasattr(phi_h5[member], 'shape'):
            if phi_h5[member].attrs['time'] == tstep: 
                ## save dataset as element of numpy array
                phidata[i,:,:,:] = phi_h5[member][...]

## get zstep for xy-plane
for i in range(zdim):
    if zdata[i,0,0]==0:
        z_xyplane = i

## get phi_xy_data
phi_xy = phidata[:, z_xyplane, :, :]

## print to screen (optional)
print('phi_xy =', phi_xy)

The next cell gets 2D meshgrid slices of xdata and ydata.

In [ ]:
## get 2D meshgrid data
xvals = xdata[z_xyplane,:,:]
yvals = ydata[z_xyplane,:,:]

## print to screen (optional)
print('xvals =', xvals)
print('yvals =', yvals)

Now plot exactly as before. (The cells used to make the color map plots are exactly as before, copied and pasted from above. The cells used to make the surface plots are slightly modifed, due to the fact that our `xvals` and `yvals` are already in meshgrid form. See comments below.)

### Color map snapshot

In [ ]:
### plot snapshot using imshow() ###

## set graphics backend
%matplotlib notebook
## import graphics libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm

## figure properties
cmap = cm.gist_rainbow
fig_width = 8
fig_height = 4
minval = np.min(phi_xy)
maxval = np.max(phi_xy)

## choose a time step
tstep = 3

## plot
time = time_steps[tstep]
fig = plt.figure(figsize=(fig_width, fig_height))
fig.suptitle('time = %.3f' % time)
ax = fig.add_subplot(111)
zi = phi_xy[tstep,:,:]
ax.imshow(zi, cmap=cmap, clim=(minval, maxval), origin='lower')
plt.show()

### Color map animation

In [ ]:
### animate using imshow() ###

## set graphics backend
%matplotlib notebook
## import graphics libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation

## figure properties
cmap = cm.gist_rainbow
fig_width = 8
fig_height = 4
minval = np.min(phi_xy)
maxval = np.max(phi_xy)

## initialize plot
plt.ioff()
time = time_steps[tstep]
fig = plt.figure(figsize=(fig_width, fig_height))
fig.suptitle('time = %.3f' % time)
ax = fig.add_subplot(111)
zi = phi_xy[0,:,:]
scene = [ax.imshow(zi, cmap=cmap, clim=(minval, maxval), origin='lower')]

## update plot
def update_plot(frame_number, time_steps, phi_xy, scene):
    time = time_steps[frame_number]
    fig.suptitle('time = %.3f' % time)
    zi = phi_xy[frame_number,:,:]
    scene[0] = ax.imshow(zi, cmap=cmap, clim=(minval, maxval), origin='lower')

## make animation
fps=5
frames=np.arange(0, len(time_steps), 1)
anim = animation.FuncAnimation(
    fig, update_plot, frames, fargs=(time_steps, phi_xy, scene), 
    interval=1000/fps, blit=False, repeat=False)
plt.close()

## play animation
from IPython.display import HTML
HTML(anim.to_html5_video())  #playback option 1
#HTML(anim.to_jshtml())       #playback option 2

### Surface plot snapshot

In [ ]:
### plot snapshot using plot_surface() ###

## set graphics backend
%matplotlib notebook
## import graphics libraries
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

## figure properties
cmap = cm.gist_rainbow
fig_width = 8
fig_height = 4
minval = np.min(phi_xy)
maxval = np.max(phi_xy)

## choose a time step
tstep = 1

## plot
fig = plt.figure(figsize=(fig_width, fig_height))
time = time_steps[tstep]
fig.suptitle('time = %.3f' % time)
ax = fig.add_subplot(111, projection='3d')
ax.set_zlim(minval, maxval)
## this was the old line which no longer works here
#xi, yi = np.meshgrid(xvals, yvals)
## this is the new way of doing things
xi = xvals #xvals is already in mesh grid form
yi = yvals #yvals is already in mesh grid form
zi = phi_xy[tstep,:,:]
ax.plot_surface(xi, yi, zi, cstride=1, rstride=1, cmap=cmap, clim=(minval, maxval))
plt.show()

### Surface plot animation

In [ ]:
### animate using plot_surface() ###

## set graphics backend
%matplotlib notebook
## import graphics libraries
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
import matplotlib.animation as animation

## figure properties
cmap = cm.gist_rainbow
fig_width = 8
fig_height = 4
minval = np.min(phi_xy)
maxval = np.max(phi_xy)

## initialize plot
plt.ioff()
fig = plt.figure(figsize=(fig_width, fig_height))
time = time_steps[0]
fig.suptitle('time = %.3f' % time)
ax = fig.add_subplot(111, projection='3d')
ax.set_zlim(minval, maxval)
## this was the old line which no longer works here
#xi, yi = np.meshgrid(xvals, yvals)
## this is the new way of doing things
xi = xvals #xvals is already in mesh grid form
yi = yvals #yvals is already in mesh grid form
zi = phi_xy[0,:,:]
scene = [ax.plot_surface(xi, yi, zi, cstride=1, rstride=1, cmap=cmap, clim=(minval, maxval))]

## update plot
def update_plot(frame_number, time_steps, phi_xy, scene):
    time = time_steps[frame_number]
    fig.suptitle('time = %.3f' % time)
    scene[0].remove()
    zi = phi_xy[frame_number,:,:]
    scene[0] = ax.plot_surface(xi, yi, zi, cstride=1, rstride=1, cmap=cmap, clim=(minval, maxval))

## make animation
fps=5
frames=np.arange(0, len(time_steps), 1)
anim = animation.FuncAnimation(
    fig, update_plot, frames, fargs=(time_steps, phi_xy, scene), 
    interval=1000/fps, blit=False, repeat=False)
plt.close()

## play animation
from IPython.display import HTML
HTML(anim.to_html5_video())  #playback option 1
#HTML(anim.to_jshtml())       #playback option 2

In [ ]:
## save animation to file
anim_file = 'wave_mol'
#anim.save(anim_file + '.mp4', writer='ffmpeg', fps=fps) # requires ffmpeg
anim.save(anim_file + '.gif', writer='imagemagick', fps=fps)

### Exercises

1) Plot phi_yz as a function of time using the data in `phi.h5`.

2) Plot phi_xz as a function of time using the data in `phi.h5`.

3) Plot phi_xy on the upper/lower z=const boundary.

4) Change the frequency of hdf5 output and replot phi_xy.

5) Plot energy in the xy plane as a function of time.